<a href="https://colab.research.google.com/github/Sai7093/colab-projects/blob/main/RSS_FEEDDSGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Install the required libraries
# We add google-generativeai for the AI features.
!pip install python-telegram-bot feedparser google-generativeai

# Step 2: Import necessary modules and set up the bot code
import asyncio
import feedparser
import html
import logging
import os
import json
from telegram import Bot
from telegram.constants import ParseMode
from telegram.error import TelegramError
import google.generativeai as genai

# --- 🔴 CONFIGURATION - REPLACE WITH YOUR NEW CREDENTIALS 🔴 ---

# Telegram Bot Token from @BotFather
BOT_TOKEN = "YOUR_NEW_BOT_TOKEN"

# Your user, group, or channel ID
CHAT_ID = "YOUR_NUMERICAL_TELEGRAM_USER_ID" # e.g., -1001234567890 for a channel

# The URLs of the RSS feeds to monitor. Add as many as you like.
RSS_FEED_URLS = [
    "https://govtjobsblog.in/feed/",
    "http://feeds.reuters.com/reuters/topNews" # Example of a second feed
]

# 🤖 AI CONFIGURATION
# Get your API key from Google AI Studio: https://aistudio.google.com/app/apikey
GEMINI_API_KEY = "YOUR_GEMINI_API_KEY"

# How often to check the feeds, in seconds. 600 seconds = 10 minutes.
CHECK_INTERVAL_SECONDS = 600

# --- END OF CONFIGURATION ---

# --- SCRIPT LOGIC (No need to edit below this line) ---

# Configure logging
logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    level=logging.INFO
)
logging.getLogger("httpx").setLevel(logging.WARNING)
logger = logging.getLogger(__name__)

# Configure the Gemini AI model
if GEMINI_API_KEY and GEMINI_API_KEY != "YOUR_GEMINI_API_KEY":
    genai.configure(api_key=GEMINI_API_KEY)
else:
    logger.warning("Gemini API Key not found. AI features will be disabled.")

# File to store the links of the last posts sent to avoid duplicates.
LAST_POSTS_FILE = "last_posts.json"

def get_last_post_links():
    """Reads the last sent post links from the JSON file."""
    if not os.path.exists(LAST_POSTS_FILE):
        return {}
    try:
        with open(LAST_POSTS_FILE, "r") as f:
            return json.load(f)
    except (json.JSONDecodeError, FileNotFoundError):
        return {}

def save_last_post_links(links_dict):
    """Saves the dictionary of last post links to the JSON file."""
    with open(LAST_POSTS_FILE, "w") as f:
        json.dump(links_dict, f, indent=4)

def find_image_in_entry(entry):
    """Tries to find an image URL within an RSS entry."""
    if 'media_content' in entry and entry.media_content:
        for media in entry.media_content:
            if 'url' in media and media.get('medium') == 'image':
                return media['url']
    if 'enclosures' in entry:
        for enclosure in entry.enclosures:
            if 'image' in enclosure.get('type', ''):
                return enclosure.href
    if 'summary' in entry:
        import re
        match = re.search(r'src="([^"]+\.(?:jpg|jpeg|png|gif))"', entry.summary)
        if match:
            return match.group(1)
    return None

async def rewrite_with_ai(entry_title, entry_summary):
    """Uses Gemini AI to rewrite content and generate tags."""
    if not GEMINI_API_KEY or GEMINI_API_KEY == "YOUR_GEMINI_API_KEY":
        logger.warning("AI rewrite skipped: Gemini API Key is not configured.")
        return None

    model = genai.GenerativeModel('gemini-1.5-flash')
    import re
    clean_summary = re.sub('<[^<]+?>', '', entry_summary)

    prompt = f"""
    You are an expert social media content creator for a job alert service.
    Your task is to rewrite the following job post to be more engaging and clear.
    Provide your output in a single, clean JSON object. Do not include any text before or after the JSON.

    The JSON object must have these exact keys: "new_title", "new_description", "tags".
    - "new_title": A catchy and informative title, under 15 words.
    - "new_description": A clear, concise summary of the job, around 50-70 words. Use emojis where appropriate.
    - "tags": An array of 3-5 relevant hashtags as strings (without the '#' symbol).

    Original Title: {entry_title}
    Original Summary: {clean_summary[:1000]}
    """

    try:
        logger.info("Sending content to AI for rewriting...")
        response = model.generate_content(prompt)
        json_text = response.text.strip().replace("```json", "").replace("```", "")
        ai_content = json.loads(json_text)

        if all(k in ai_content for k in ["new_title", "new_description", "tags"]):
            logger.info("AI rewrite successful.")
            return ai_content
        else:
            logger.error("AI response was missing required keys.")
            return None
    except Exception as e:
        logger.error(f"An error occurred during AI processing: {e}", exc_info=True)
        return None

async def check_all_feeds_and_send_updates(bot: Bot):
    """Iterates through all RSS feeds, checks for new entries, and sends updates."""
    logger.info(f"Starting feed check for {len(RSS_FEED_URLS)} feeds.")
    last_post_links = get_last_post_links()

    for feed_url in RSS_FEED_URLS:
        try:
            logger.info(f"Checking RSS feed: {feed_url}")
            feed = feedparser.parse(feed_url)

            if feed.bozo:
                logger.error(f"Error parsing feed {feed_url}: {feed.bozo_exception}")
                continue
            if not feed.entries:
                logger.warning(f"Feed {feed_url} has no entries.")
                continue

            latest_entry = feed.entries[0]
            last_sent_link = last_post_links.get(feed_url)

            if latest_entry.link != last_sent_link:
                logger.info(f"New post found in {feed_url}: {latest_entry.title}")

                ai_data = await rewrite_with_ai(latest_entry.title, latest_entry.summary)

                if ai_data:
                    title = ai_data['new_title']
                    description = ai_data['new_description']
                    tags = " ".join([f"#{tag}" for tag in ai_data['tags']])
                else:
                    logger.warning("AI processing failed. Falling back to original content.")
                    title = html.escape(latest_entry.title)
                    description = "Read more about this opportunity at the link below."
                    tags = "#News #Update"

                signature = (
                    f"\n\n{tags}\n\n"
                    "----\n"
                    "✍️ *Rewritten by MY SAI*\n"
                    "🔗 [Original Article]({link})"
                ).format(link=latest_entry.link)

                final_caption = f"📢 *{title}*\n\n{description}{signature}"

                image_url = find_image_in_entry(latest_entry)

                if image_url:
                    logger.info(f"Found image: {image_url}")
                    await bot.send_photo(
                        chat_id=CHAT_ID,
                        photo=image_url,
                        caption=final_caption,
                        parse_mode=ParseMode.MARKDOWN
                    )
                else:
                    logger.info("No image found, sending text message.")
                    await bot.send_message(
                        chat_id=CHAT_ID,
                        text=final_caption,
                        parse_mode=ParseMode.MARKDOWN,
                        disable_web_page_preview=True
                    )

                logger.info(f"Successfully sent message for post: {latest_entry.title}")
                last_post_links[feed_url] = latest_entry.link
            else:
                logger.info(f"No new posts in {feed_url}.")

        except Exception as e:
            logger.error(f"An error occurred while processing feed {feed_url}: {e}", exc_info=True)
            continue

    save_last_post_links(last_post_links)
    logger.info("Feed check cycle complete.")

async def main():
    """Main function to start the bot and schedule the feed check."""
    bot = Bot(token=BOT_TOKEN)
    logger.info("AI-powered Multi-RSS Bot started. This cell will run indefinitely.")

    try:
        await bot.send_message(chat_id=CHAT_ID, text="✅ AI-Powered Multi-RSS Feed Bot is online and monitoring from Google Colab.")
    except TelegramError as e:
        logger.error(f"Failed to send startup message to CHAT_ID '{CHAT_ID}'. "
                     f"Please ensure the ID is correct and that you have started a chat with the bot. Error: {e}")
        return

    while True:
        await check_all_feeds_and_send_updates(bot)
        logger.info(f"Waiting for {CHECK_INTERVAL_SECONDS} seconds until next check.")
        await asyncio.sleep(CHECK_INTERVAL_SECONDS)

# --- Step 3: Run the bot ---
# This will start the bot. The cell will continue to run until you manually stop it.
try:
    await main()
except Exception as e:
    logger.critical(f"Bot exited with a critical error: {e}", exc_info=True)
